<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# LightGBM: A Highly Efficient Gradient Boosting Decision Tree
This notebook will give you an example of how to train a LightGBM model to estimate click-through rates on an e-commerce advertisement. We will train a LightGBM based model on the Criteo dataset.

[LightGBM](https://github.com/Microsoft/LightGBM) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:
* Fast training speed and high efficiency.
* Low memory usage.
* Great accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

## Global Settings and Imports

In [ ]:
import os
import sys
import numpy as np
import lightgbm as lgb
import category_encoders as ce
from tempfile import TemporaryDirectory
from sklearn.metrics import roc_auc_score, log_loss

import recommenders.datasets.criteo as criteo
import recommenders.models.lightgbm.lightgbm_utils as lgb_utils
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("LightGBM version: {}".format(lgb.__version__))

System version: 3.11.9 (main, Apr 19 2024, 16:48:06) [GCC 11.2.0]
LightGBM version: 4.4.0


### Parameter Setting
Let's set the main related parameters for LightGBM now. Basically, the task is a binary classification (predicting click or no click), so the objective function is set to binary logloss, and 'AUC' metric, is used as a metric which is less effected by imbalance in the classes of the dataset.

Generally, we can adjust the number of leaves (MAX_LEAF), the minimum number of data in each leaf (MIN_DATA), maximum number of trees (NUM_OF_TREES), the learning rate of trees (TREE_LEARNING_RATE) and EARLY_STOPPING_ROUNDS (to avoid overfitting) in the model to get better performance.

Besides, we can also adjust some other listed parameters to optimize the results. [In this link](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst), a list of all the parameters is shown. Also, some advice on how to tune these parameters can be found [in this url](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst). 

In [2]:
MAX_LEAF = 64
MIN_DATA = 20
NUM_OF_TREES = 100
TREE_LEARNING_RATE = 0.15
EARLY_STOPPING_ROUNDS = 20
METRIC = "auc"
SIZE = "sample"

In [3]:
params = {
    "task": "train",
    "boosting_type": "gbdt",
    "num_class": 1,
    "objective": "binary",
    "metric": METRIC,
    "num_leaves": MAX_LEAF,
    "min_data": MIN_DATA,
    "boost_from_average": True,
    # set it according to your cpu cores.
    "num_threads": 20,
    "feature_fraction": 0.8,
    "learning_rate": TREE_LEARNING_RATE,
}

## Data Preparation
Here we use CSV format as the example data input. Our example data is a sample (about 100 thousand samples) from [Criteo dataset](https://www.kaggle.com/c/criteo-display-ad-challenge). The Criteo dataset is a well-known industry benchmarking dataset for developing CTR prediction models, and it's frequently adopted as evaluation dataset by research papers. The original dataset is too large for a lightweight demo, so we sample a small portion from it as a demo dataset.

Specifically, there are 39 columns of features in Criteo, where 13 columns are numerical features (I1-I13) and the other 26 columns are categorical features (C1-C26).

In [4]:
nume_cols = ["I" + str(i) for i in range(1, 14)]
cate_cols = ["C" + str(i) for i in range(1, 27)]
label_col = "Label"

header = [label_col] + nume_cols + cate_cols
with TemporaryDirectory() as tmp:
    all_data = criteo.load_pandas_df(size=SIZE, local_cache_path=tmp, header=header)
display(all_data.head())

100%|██████████| 8.58k/8.58k [00:00<00:00, 10.4kKB/s]


Label   I1   I2    I3    I4      I5    I6    I7   I8     I9  ...       C17  \
0      0  1.0    1   5.0   0.0  1382.0   4.0  15.0  2.0  181.0  ...  e5ba7672   
1      0  2.0    0  44.0   1.0   102.0   8.0   2.0  2.0    4.0  ...  07c540c4   
2      0  2.0    0   1.0  14.0   767.0  89.0   4.0  2.0  245.0  ...  8efede7f   
3      0  NaN  893   NaN   NaN  4392.0   NaN   0.0  0.0    0.0  ...  1e88c74f   
4      0  3.0   -1   NaN   0.0     2.0   0.0   3.0  0.0    0.0  ...  1e88c74f   

        C18       C19       C20       C21       C22       C23       C24  \
0  f54016b9  21ddcdc9  b1252a9d  07b5194c       NaN  3a171ecb  c5c50484   
1  b04e4670  21ddcdc9  5840adea  60f6221e       NaN  3a171ecb  43f13e8b   
2  3412118d       NaN       NaN  e587c466  ad3062eb  3a171ecb  3b183c5c   
3  74ef3502       NaN       NaN  6b3a5ca6       NaN  3a171ecb  9117a34a   
4  26b3c7a7       NaN       NaN  21c9516a       NaN  32c7478e  b34f3128   

        C25       C26  
0  e8b83407  9727dd16  
1  e8b83407  731c3655  
2       NaN       NaN  
3       NaN       NaN  
4       NaN       NaN  

[5 rows x 40 columns]

First, we cut three sets (train_data (first 80%), valid_data (middle 10%) and test_data (last 10%)), cut from the original all data. <br>
Notably, considering the Criteo is a kind of time-series streaming data, which is also very common in recommendation scenario, we split the data by its order.

In [5]:
# split data to 3 sets    
length = len(all_data)
train_data = all_data.loc[:0.8*length-1]
valid_data = all_data.loc[0.8*length:0.9*length-1]
test_data = all_data.loc[0.9*length:]

## Basic Usage
### Ordinal Encoding
Considering LightGBM could handle the low-frequency features and missing value by itself, for basic usage, we only encode the string-like categorical features by an ordinal encoder.

In [6]:
ord_encoder = ce.ordinal.OrdinalEncoder(cols=cate_cols)

def encode_csv(df, encoder, label_col, typ="fit"):
    if typ == "fit":
        df = encoder.fit_transform(df)
    else:
        df = encoder.transform(df)
    y = df[label_col].values
    del df[label_col]
    return df, y

train_x, train_y = encode_csv(train_data, ord_encoder, label_col)
valid_x, valid_y = encode_csv(valid_data, ord_encoder, label_col, "transform")
test_x, test_y = encode_csv(test_data, ord_encoder, label_col, "transform")

print("Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n"
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))
train_x.head()

Train Data Shape: X: (80000, 39); Y: (80000,).
Valid Data Shape: X: (10000, 39); Y: (10000,).
Test Data Shape: X: (10000, 39); Y: (10000,).



I1   I2    I3    I4      I5    I6    I7   I8     I9  I10  ...  C17  C18  \
0  1.0    1   5.0   0.0  1382.0   4.0  15.0  2.0  181.0  1.0  ...    1    1   
1  2.0    0  44.0   1.0   102.0   8.0   2.0  2.0    4.0  1.0  ...    2    2   
2  2.0    0   1.0  14.0   767.0  89.0   4.0  2.0  245.0  1.0  ...    3    3   
3  NaN  893   NaN   NaN  4392.0   NaN   0.0  0.0    0.0  NaN  ...    4    4   
4  3.0   -1   NaN   0.0     2.0   0.0   3.0  0.0    0.0  1.0  ...    4    5   

    C19  C20  C21  C22  C23  C24  C25   C26  
0     1    1    1   11    1    1    1     1  
1     1    2    2   11    1    2    1     2  
2  1222    4    3    1    1    3   49  8199  
3  1222    4    4   11    1    4   49  8199  
4  1222    4    5   11    2    5   49  8199  

[5 rows x 39 columns]

### Create model
When both hyper-parameters and data are ready, we can create a model:

In [7]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params, categorical_feature=cate_cols)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_test = lgb.Dataset(test_x, test_y.reshape(-1), reference=lgb_train, categorical_feature=cate_cols)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      valid_sets=lgb_valid,
                      callbacks=[lgb.early_stopping(EARLY_STOPPING_ROUNDS)])


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 17958, number of negative: 62042
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027934 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 38971
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.224475 -> initscore=-1.239776
[LightGBM] [Info] Start training from score -1.239776
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[18]	valid_0's auc: 0.759658


Now let's see what is the model's performance:

In [8]:
test_preds = lgb_model.predict(test_x)
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
res_basic = {"auc": auc, "logloss": logloss}
print(res_basic)


{'auc': 0.7655408801711783, 'logloss': 0.4682583178835999}


In [9]:
# Record results for tests - ignore this cell
store_metadata("auc_basic", res_basic["auc"])
store_metadata("logloss_basic", res_basic["logloss"])

<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>
## Optimized Usage
### Label-encoding and Binary-encoding
Next, since LightGBM has a better capability in handling dense numerical features effectively, we try to convert all the categorical features in original data into numerical ones, by label-encoding [3] and binary-encoding [4]. Also due to the sequence property of Criteo, the label-encoding we adopted is executed one-by-one, which means we encode the samples in order, by the information of the previous samples before each sample (sequential label-encoding and sequential count-encoding). Besides, we also filter the low-frequency categorical features and fill the missing values by the mean of corresponding columns for the numerical features. (consulting `lgb_utils.NumEncoder`)

Specifically, in `lgb_utils.NumEncoder`, the main steps are as follows.
* Firstly, we convert the low-frequency categorical features to `"LESS"` and the missing categorical features to `"UNK"`. 
* Secondly, we convert the missing numerical features into the mean of corresponding columns. 
* Thirdly, the string-like categorical features are ordinal encoded like the example shown in basic usage. 
* And then, we target encode the categorical features in the samples order one-by-one. For each sample, we add the label and count information of its former samples into the data and produce new features. Formally, for $i=1,2,...,n$, we add $\frac{\sum\nolimits_{j=1}^{i-1} I(x_j=c) \cdot y}{\sum\nolimits_{j=1}^{i-1} I(x_j=c)}$ as a new label feature for current sample $x_i$, where $c$ is a category to encode in current sample, so $(i-1)$ is the number of former samples, and $I(\cdot)$ is the indicator function that check the former samples contain $c$ (whether $x_j=c$) or not. At the meantime, we also add the count frequency of $c$, which is $\frac{\sum\nolimits_{j=1}^{i-1} I(x_j=c)}{i-1}$, as a new count feature. 
* Finally, based on the results of ordinal encoding, we add the binary encoding results as new columns into the data.

Note that the statistics used in the above process only updates when fitting the training set, while maintaining static when transforming the testing set because the label of test data should be considered as unknown.

In [10]:
label_col = "Label"
num_encoder = lgb_utils.NumEncoder(cate_cols, nume_cols, label_col)
train_x, train_y = num_encoder.fit_transform(train_data)
valid_x, valid_y = num_encoder.transform(valid_data)
test_x, test_y = num_encoder.transform(test_data)
del num_encoder
print("Train Data Shape: X: {trn_x_shape}; Y: {trn_y_shape}.\nValid Data Shape: X: {vld_x_shape}; Y: {vld_y_shape}.\nTest Data Shape: X: {tst_x_shape}; Y: {tst_y_shape}.\n"
      .format(trn_x_shape=train_x.shape,
              trn_y_shape=train_y.shape,
              vld_x_shape=valid_x.shape,
              vld_y_shape=valid_y.shape,
              tst_x_shape=test_x.shape,
              tst_y_shape=test_y.shape,))


2025-03-31 22:23:47,402 [INFO] Filtering and fillna features
100%|██████████| 13/13 [00:00<00:00, 175.39it/s]
2025-03-31 22:23:52,763 [INFO] Ordinal encoding cate features
2025-03-31 22:23:54,739 [INFO] Target encoding cate features
100%|██████████| 26/26 [00:06<00:00,  4.23it/s]
2025-03-31 22:24:00,890 [INFO] Start manual binary encoding
100%|██████████| 26/26 [00:05<00:00,  4.91it/s]
2025-03-31 22:24:14,799 [INFO] Filtering and fillna features
100%|██████████| 13/13 [00:00<00:00, 1011.14it/s]
2025-03-31 22:24:15,061 [INFO] Ordinal encoding cate features
2025-03-31 22:24:15,171 [INFO] Target encoding cate features
100%|██████████| 26/26 [00:00<00:00, 38.43it/s]
2025-03-31 22:24:15,853 [INFO] Start manual binary encoding
100%|██████████| 26/26 [00:04<00:00,  6.37it/s]
2025-03-31 22:24:29,249 [INFO] Filtering and fillna features
100%|██████████| 13/13 [00:00<00:00, 939.65it/s]
2025-03-31 22:24:29,773 [INFO] Ordinal encoding cate features
2025-03-31 22:24:30,059 [INFO] Target encoding ca

Train Data Shape: X: (80000, 268); Y: (80000, 1).
Valid Data Shape: X: (10000, 268); Y: (10000, 1).
Test Data Shape: X: (10000, 268); Y: (10000, 1).



### Training and Evaluation

In [11]:
lgb_train = lgb.Dataset(train_x, train_y.reshape(-1), params=params)
lgb_valid = lgb.Dataset(valid_x, valid_y.reshape(-1), reference=lgb_train)
lgb_model = lgb.train(params,
                      lgb_train,
                      num_boost_round=NUM_OF_TREES,
                      valid_sets=lgb_valid,
                      callbacks=[lgb.early_stopping(EARLY_STOPPING_ROUNDS)])

[LightGBM] [Info] Number of positive: 17958, number of negative: 62042
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.235469 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15787
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 267
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.224475 -> initscore=-1.239776
[LightGBM] [Info] Start training from score -1.239776
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[43]	valid_0's auc: 0.77085


In [12]:
test_preds = lgb_model.predict(test_x)

In [13]:
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
res_optim = {"auc": auc, "logloss": logloss}

print(res_optim)

{'auc': 0.7758548016657666, 'logloss': 0.46030887404896165}


In [14]:
# Record results for tests - ignore this cell
store_metadata("auc_opt", res_optim["auc"])
store_metadata("logloss_opt", res_optim["logloss"])

## Model saving and loading
Now we finish the basic training and testing for LightGBM, next let's try to save and reload the model, and then evaluate it again.

In [15]:
with TemporaryDirectory() as tmp:
    save_file = os.path.join(tmp, "finished.model")
    lgb_model.save_model(save_file)
    loaded_model = lgb.Booster(model_file=save_file)

# eval the performance again
test_preds = loaded_model.predict(test_x)


In [16]:
auc = roc_auc_score(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))
logloss = log_loss(np.asarray(test_y.reshape(-1)), np.asarray(test_preds))

print({"auc": auc, "logloss": logloss})

{'auc': 0.7758548016657666, 'logloss': 0.46030887404896165}


## Additional Reading

\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154.<br>
\[2\] The parameters of LightGBM: https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst <br>
\[3\] Anna Veronika Dorogush, Vasily Ershov, and Andrey Gulin. 2018. CatBoost: gradient boosting with categorical features support. arXiv preprint arXiv:1810.11363 (2018).<br>
\[4\] Scikit-learn. 2018. categorical_encoding. https://github.com/scikit-learn-contrib/categorical-encoding<br>
